# Lesson 3: Structured Output & Context

In this notebook, we'll learn how to:

1. **Force structured output** - Make agents return typed Pydantic models instead of free-form text
2. **Use RunContextWrapper** - Inject shared state (user info, DB connections) into tools

## Why Structured Output?

Free-form text responses are:
- Hard to parse programmatically
- Inconsistent in format
- Error-prone when extracting data

Structured output gives you:
- Type-safe responses
- Automatic validation
- Easy integration with downstream systems

## Setup

In [1]:
import nest_asyncio
nest_asyncio.apply()

import os
import getpass

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("OpenAI API Key: ")

## Basic Structured Output with `output_type`

Use the `output_type` parameter to force the agent to return a Pydantic model.

In [2]:
from agents import Agent, Runner
from pydantic import BaseModel, Field

class SupportTicket(BaseModel):
    """A classified support ticket."""
    priority: str = Field(description="Priority level: low, medium, high, or critical")
    department: str = Field(description="Department: sales, support, billing, or technical")
    sentiment: str = Field(description="Customer sentiment: positive, neutral, or negative")
    summary: str = Field(description="Brief summary of the issue")

classifier = Agent(
    name="TicketClassifier",
    instructions="""You classify support tickets. Analyze the customer message and extract:
    - Priority (based on urgency and impact)
    - Department (who should handle this)
    - Sentiment (how the customer feels)
    - Summary (brief description)""",
    model="gpt-4.1",
    output_type=SupportTicket  # <-- Forces structured output
)

In [3]:
# Classify a ticket
result = await Runner.run(
    classifier,
    "I've been charged twice for my subscription! This is unacceptable. I need a refund immediately!"
)

# Access the structured output
ticket = result.final_output
print(f"Priority: {ticket.priority}")
print(f"Department: {ticket.department}")
print(f"Sentiment: {ticket.sentiment}")
print(f"Summary: {ticket.summary}")

Priority: high
Department: billing
Sentiment: negative
Summary: Customer was double charged for subscription and demands an immediate refund.


In [4]:
# The output is a proper Pydantic model
print(f"\nType: {type(ticket)}")
print(f"\nAs dict: {ticket.model_dump()}")


Type: <class '__main__.SupportTicket'>

As dict: {'priority': 'high', 'department': 'billing', 'sentiment': 'negative', 'summary': 'Customer was double charged for subscription and demands an immediate refund.'}


## Nested Models and Collections

You can use complex Pydantic models with nested types.

In [5]:
from typing import List

class ActionItem(BaseModel):
    """An action item extracted from meeting notes."""
    task: str = Field(description="The task to be done")
    assignee: str = Field(description="Person responsible")
    due_date: str = Field(description="Due date if mentioned, otherwise 'TBD'")

class MeetingNotes(BaseModel):
    """Structured meeting notes."""
    title: str = Field(description="Meeting title or topic")
    key_decisions: List[str] = Field(description="Key decisions made")
    action_items: List[ActionItem] = Field(description="Action items with assignees")

note_taker = Agent(
    name="NoteTaker",
    instructions="Extract structured notes from meeting transcripts.",
    model="gpt-4.1",
    output_type=MeetingNotes
)

In [6]:
transcript = """
Project sync meeting. We decided to launch the beta next month. 
Sarah will finalize the UI by Friday. 
Mike needs to set up the deployment pipeline by end of week.
We agreed to skip the mobile version for now and focus on web first.
"""

result = await Runner.run(note_taker, transcript)
notes = result.final_output

print(f"Title: {notes.title}")
print(f"\nKey Decisions:")
for decision in notes.key_decisions:
    print(f"  - {decision}")
print(f"\nAction Items:")
for item in notes.action_items:
    print(f"  - {item.task} ({item.assignee}, due: {item.due_date})")

Title: Project Sync Meeting

Key Decisions:
  - Launch the beta next month.
  - Skip the mobile version for now and focus on web first.

Action Items:
  - Finalize the UI (Sarah, due: Friday)
  - Set up the deployment pipeline (Mike, due: end of week)


## Converting to DataFrame

Structured output integrates nicely with pandas.

In [7]:
import pandas as pd

# Convert action items to DataFrame
df = pd.DataFrame([item.model_dump() for item in notes.action_items])
df

,task,assignee,due_date
0,Finalize the UI,Sarah,Friday
1,Set up the deployment pipeline,Mike,end of week


# how does the LLM - function calling handle errors or exceptions? 

# sorry, Lucas, the structured output is a json or does it instantiate an object of the type?

## RunContextWrapper: Injecting Shared State

Often your tools need access to shared state:
- User information (name, ID, permissions)
- Database connections
- API clients
- Configuration

`RunContextWrapper` lets you inject this state without passing it through the LLM.

**Important**: The context object is **not** sent to the LLM. It's purely local.

In [10]:
from dataclasses import dataclass
from agents import Agent, Runner, RunContextWrapper, function_tool

@dataclass
class UserContext:
    """Context about the current user."""
    user_id: str
    user_name: str
    account_type: str  # 'free' or 'premium'

@function_tool
async def get_user_discounts(wrapper: RunContextWrapper[UserContext]) -> str:
    """Get available discounts for the current user."""
    ctx = wrapper.context
    if ctx.account_type == "premium":
        return f"{ctx.user_name}, as a premium member you get: 20% off all items, free shipping, early access to sales."
    else:
        return f"{ctx.user_name}, upgrade to premium to unlock discounts! Current offer: First month 50% off."

@function_tool
async def get_order_history(wrapper: RunContextWrapper[UserContext]) -> str:
    """Get the user's recent orders."""
    ctx = wrapper.context
    # Simulated order history based on user
    return f"Recent orders for {ctx.user_name} (ID: {ctx.user_id}): Order #123 - Laptop, Order #124 - Mouse"

In [11]:
support_agent = Agent(
    name="SupportAgent",
    instructions="""You are a customer support agent. 
    Use the available tools to help customers with their inquiries.
    Be friendly and personalized in your responses.""",
    model="gpt-4.1",
    tools=[get_user_discounts, get_order_history]
)

In [ ]:
import asyncio

# Create context for a premium user
premium_user = UserContext(
    user_id="usr_123",
    user_name="Alice",
    account_type="premium"
)

async def help_premium_user():
    result = await Runner.run(
        support_agent,
        "What discounts do I have available?",
        context=premium_user  # <-- Pass context here
    )
    print(result.final_output)

asyncio.run(help_premium_user())

As a premium member, you have some great discounts and perks available:

- 20% off all items
- Free shipping on your orders
- Early access to sales

If you need details about specific items or want to know how to apply these discounts, just let me know!


In [14]:
# Now with a free user
free_user = UserContext(
    user_id="usr_456",
    user_name="Bob",
    account_type="free"
)

async def help_free_user():
    result = await Runner.run(
        support_agent,
        "What discounts do I have?",
        context=free_user
    )
    print(result.final_output)

asyncio.run(help_free_user())

Bob, it looks like you don't have any regular discounts available at the moment. However, there's a special offer for you: if you upgrade to premium, you'll get your first month at 50% off!

If you have any questions about this offer or want more details on how to upgrade, just let me know—I'm happy to help!


Notice how the same agent gives different responses based on the context - without the LLM ever seeing the raw context object!

## Combining Structured Output and Context

Let's build a more complete example: a ticket classifier that's aware of user context.

In [15]:
class ClassifiedTicket(BaseModel):
    """A ticket with classification and routing info."""
    priority: str
    department: str
    auto_escalate: bool = Field(description="True if this should be auto-escalated")
    suggested_response: str

@dataclass
class CustomerContext:
    customer_id: str
    is_vip: bool
    lifetime_value: float
    open_tickets: int

@function_tool
async def get_customer_info(wrapper: RunContextWrapper[CustomerContext]) -> str:
    """Get information about the current customer."""
    ctx = wrapper.context
    vip_status = "VIP Customer" if ctx.is_vip else "Standard Customer"
    return f"{vip_status}, Lifetime Value: ${ctx.lifetime_value:.2f}, Open Tickets: {ctx.open_tickets}"

smart_classifier = Agent(
    name="SmartClassifier",
    instructions="""Classify support tickets. Consider customer context:
    - VIP customers or high lifetime value → higher priority
    - Multiple open tickets → potential escalation
    - Auto-escalate if VIP with negative sentiment""",
    model="gpt-4.1",
    tools=[get_customer_info],
    output_type=ClassifiedTicket
)

In [16]:
# VIP customer with a complaint
vip_context = CustomerContext(
    customer_id="cust_vip_001",
    is_vip=True,
    lifetime_value=50000.00,
    open_tickets=2
)

async def classify_vip_ticket():
    result = await Runner.run(
        smart_classifier,
        "Your service has been terrible lately! I've been waiting 3 days for a response!",
        context=vip_context
    )
    ticket = result.final_output
    print(f"Priority: {ticket.priority}")
    print(f"Department: {ticket.department}")
    print(f"Auto-escalate: {ticket.auto_escalate}")
    print(f"Suggested response: {ticket.suggested_response}")

asyncio.run(classify_vip_ticket())

Priority: High
Department: Customer Success
Auto-escalate: True
Suggested response: We're very sorry for the delay and for the frustration this has caused you. As a valued VIP customer, your concerns are our top priority. We're escalating your ticket now to ensure a prompt resolution. Thank you for your patience, and we appreciate your continued trust in us.


In [17]:
# Regular customer with a simple question
regular_context = CustomerContext(
    customer_id="cust_reg_042",
    is_vip=False,
    lifetime_value=150.00,
    open_tickets=0
)

async def classify_regular_ticket():
    result = await Runner.run(
        smart_classifier,
        "How do I reset my password?",
        context=regular_context
    )
    ticket = result.final_output
    print(f"Priority: {ticket.priority}")
    print(f"Department: {ticket.department}")
    print(f"Auto-escalate: {ticket.auto_escalate}")
    print(f"Suggested response: {ticket.suggested_response}")

asyncio.run(classify_regular_ticket())

Priority: Low
Department: Support
Auto-escalate: False
Suggested response: To reset your password, please click on the 'Forgot Password' link on the login page and follow the instructions. If you need further assistance, let us know!


## Key Takeaways

1. **`output_type`** forces agents to return structured Pydantic models
2. **Nested models** work great - use `List[Model]` for collections
3. **`RunContextWrapper`** injects local state into tools without exposing it to the LLM
4. **Context + Structured Output** = powerful pattern for context-aware, type-safe agents
5. **Type hints** in context (`RunContextWrapper[UserContext]`) ensure type safety

Next up: **Multi-Agent Patterns** - handoffs and agents-as-tools.